In [7]:
# load module and pre generate data
from graph_tree_link_module import *

import numpy as np

import pickle

# Open the file in binary read mode and load the object
with open("graph_tree.pkl", "rb") as file:
    gt = pickle.load(file)


In [ ]:
sliders_data = [{"id": gt.attr_name[i], "min": gt.X_maxmin[1][i], "max": gt.X_maxmin[0][i], "value": gt.root.center[i], "type": "f"} for i in range(len(gt.attr_name))]
# print(sliders_data)

recom_data = [{"i": i, "id": gt.attr_name[i], "max_val": gt.recom[i][0].center[i], "min_val": gt.recom[i][1].center[i]} for i in range(len(gt.attr_name))]
# print(recom_data)

status = gt.get_suround_cluster(gt.root, gt.root.center)
print(status)
def get_suround_node_data(status):
    cur_node = status["cur_cluster"]
    cur_cluster = [{"id": cur_node.id, "position": "center", "x": 300, "y": 200, "color": "red"}]

    contra_node = status["contra_cluster"]
    contra_cluster = [{"id": contra_node[i].id, "position": "left", "x": 100, "y": 150+i*100, "color": "green"} for i in range(len(contra_node))]

    super_node = status["super_cluster"]
    super_cluster = [{"id": super_node.id, "position": "top", "x": 300, "y": 100, "color": "dark red"}] if super_node else []

    sim_node = status["neibour_cluster"]
    sim_cluster = [{"id": sim_node[i].id, "position": "right", "x": 500, "y": 200+i*100, "color": "blue"} for i in range(len(sim_node))] if sim_node else []

    sub_node = status["sub_cluster"]
    sub_cluster = [{"id": sub_node[i].id, "position": "bottom", "x": 200+i*100, "y": 350, "color": "pink"} for i in range(len(sub_node))] if sub_node else []

    return cur_cluster + contra_cluster + super_cluster + sim_cluster + sub_cluster

graph_data = get_suround_node_data(status)

print(graph_data)

tree_data = {
        "name": "Root",
        "children": [
            {
                "name": "Child 1",
                "children": [
                    {"name": "Grandchild 1"},
                    {"name": "Grandchild 2"}
                ]
            },
            {
                "name": "Child 2",
                "children": [
                    {"name": "Grandchild 3"},
                    {"name": "Grandchild 4"}
                ]
            }
        ]
    }

In [ ]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

# slider ####################################################################
@app.route('/sliders')
def sliders():
    return jsonify(sliders_data)

@app.route('/handle_slider', methods=['POST'])
def submit_slider_values():
    data = request.json
    print('Received slider values:', data)
    query = np.array([float(val) for val in data.values()])
    cur = gt.search(query)
    print(cur.id)
    print(gt.status)
    global graph_data
    graph_data = get_suround_node_data(gt.status)
    # Process the data as needed
    return jsonify({'status': 'slider respon success', 'received': data})
# slider end ################################################################

# recom ####################################################################
@app.route('/recom')
def recom():
    return jsonify(recom_data)

@app.route('/handle_recom', methods=['POST'])
def handle_recom():
    data = request.json
    button_id = int(data.get('buttonId'))
    val = int(data.get('val'))
    print(button_id, val)
    moved_node = gt.recom[button_id][val]
    gt.get_suround_cluster(moved_node)
    global graph_data
    graph_data = get_suround_node_data(gt.status)
    # Handle the button click based on the button_id
    return jsonify({'status': 'recom respon success', 'buttonId': button_id, 'val': val, "data": graph_data})
# recom end ####################################################################

@app.route('/graph-data')
def graph():
    return jsonify(graph_data)

@app.route('/tree-data')
def tree():
    return jsonify(tree_data)

if __name__ == '__main__':
    app.run(port=8123)
